In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

25/03/02 20:07:15 WARN Utils: Your hostname, radu-ai resolves to a loopback address: 127.0.1.1; using 192.168.1.251 instead (on interface enp47s0)
25/03/02 20:07:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/02 20:07:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.5.4'

In [5]:
df_yellow = spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [15]:
df_yellow.createOrReplaceTempView('yellow')

In [31]:
df_yellow.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', LongType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('Airport_fee', DoubleType(), True)])

In [45]:
df_taxi_zone_lookup = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

In [47]:
df_taxi_zone_lookup.createOrReplaceTempView('taxi_zone_lookup')

In [49]:
df_taxi_zone_lookup.schema

StructType([StructField('LocationID', StringType(), True), StructField('Borough', StringType(), True), StructField('Zone', StringType(), True), StructField('service_zone', StringType(), True)])

In [33]:
spark.sql("""
SELECT 
    COUNT(*) AS count_trips
FROM
    yellow
WHERE 
    DATE(tpep_pickup_datetime) = '2024-10-15'
""").show()

+-----------+
|count_trips|
+-----------+
|     128893|
+-----------+



In [35]:
spark.sql("""
SELECT 
    MAX(TIMESTAMPDIFF(SECOND, tpep_pickup_datetime, tpep_dropoff_datetime)) / 3600 AS longest_trip_hours
FROM 
    yellow
""").show()

+------------------+
|longest_trip_hours|
+------------------+
|162.61777777777777|
+------------------+



In [57]:
spark.sql("""
SELECT 
    tzl.Zone AS least_frequent_pickup_zone,
    COUNT(y.PULocationID) AS pickup_count
FROM yellow y
JOIN taxi_zone_lookup tzl
    ON y.PULocationID = tzl.LocationID
GROUP BY tzl.Zone
ORDER BY pickup_count ASC
LIMIT 1
""").show()

+--------------------------+------------+
|least_frequent_pickup_zone|pickup_count|
+--------------------------+------------+
|      Governor's Island...|           1|
+--------------------------+------------+



In [7]:
df_yellow \
    .repartition(4) \
    .write.parquet('data/homework/yellow', mode='overwrite')